Using: udpipe, nltk, pymorphy2, scipy and set methods (intersection, difference, union)

# Обрабатываем корпус UDPipe

Обработка через командную строку: udpipe -input horizontal --output conllu --tokenize --tag --parse russian-syntagrus-ud-2.4-190531.udpipe testset2.txt --outfile=testset2.connlu

# Извлечение глаголов, встречающихся более 50 раз

In [0]:
from nltk.parse import DependencyGraph

In [0]:
trees = []

with open('testset2.conllu') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [44]:
print(trees[0])

1	"	"	PUNCT	_	_	3	punct	_	SpacesBefore=\n\t|SpaceAfter=No
2	Я	я	PRON	_	Case=Nom|Number=Sing|Person=1	3	nsubj	_	_
3	предложил	предложить	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
4	бы	бы	AUX	_	_	3	aux	_	_
5	"	"	PUNCT	_	_	3	punct	_	SpaceAfter=No
6	Единой	единый	ADJ	_	Case=Gen|Degree=Pos|Gender=Fem|Number=Sing	7	amod	_	_
7	России	Россия	PROPN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	9	obl	_	SpaceAfter=No
8	"	"	PUNCT	_	_	9	punct	_	_
9	объединить	объединить	VERB	_	Aspect=Perf|VerbForm=Inf|Voice=Act	3	xcomp	_	_
10	в	в	ADP	_	_	12	case	_	_
11	одном	один	NUM	_	Case=Loc|Gender=Neut	12	nummod	_	_
12	окне	окно	NOUN	_	Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing	9	obl	_	_
13	не	не	PART	_	_	14	advmod	_	_
14	только	только	PART	_	_	15	advmod	_	_
15	депутатов	депутат	NOUN	_	Animacy=Anim|Case=Gen|Gender=Masc|Number=Plur	9	obl	_	SpaceAfter=No
16	,	,	PUNCT	_	_	19	punct	_	_
17	но	но	CCONJ	_	_	19	cc	_	_
18	и	и	PART	_	_	17	advmod	_	_
19	суд	суд	NO

Извлечём все глаголы

In [0]:
all_verbs = []
for tree in trees:
  for t in tree.split('\n'):
    line = t.split('\t')
    if len(line) > 3:
      if line[3] == 'VERB':
        all_verbs.append(line[2])

In [0]:
from collections import Counter

In [0]:
c = Counter(all_verbs)

In [58]:
c.most_common(3)

[('обвинить', 461), ('мочь', 349), ('признать', 345)]

Уберём глаголы, встречающиеся менее 50 раз

In [0]:
verbs = [v for v in c.elements() if c[v] > 50]

In [0]:
verbs = set(Counter(verbs))

In [67]:
verbs

{'арестовывать',
 'быть',
 'взыскать',
 'вынести',
 'выплатить',
 'говорить',
 'дать',
 'делать',
 'доказать',
 'запретить',
 'заявить',
 'иметь',
 'использовать',
 'мочь',
 'назначить',
 'напомнить',
 'направить',
 'находиться',
 'начаться',
 'нет',
 'обвинить',
 'обвинять',
 'обжаловать',
 'обратиться',
 'объявить',
 'обязать',
 'оказаться',
 'оспаривать',
 'оспорить',
 'оставить',
 'отказать',
 'отказаться',
 'отклонить',
 'отменить',
 'передать',
 'подать',
 'подтвердить',
 'получить',
 'постановить',
 'пояснить',
 'предъявить',
 'приговорить',
 'признать',
 'принадлежать',
 'принять',
 'провести',
 'просить',
 'пытаться',
 'рассказать',
 'рассматривать',
 'рассмотреть',
 'решить',
 'связать',
 'сказать',
 'согласиться',
 'сообщить',
 'стать',
 'счесть',
 'считать',
 'требовать',
 'удаться',
 'удовлетворить',
 'утверждать',
 'являться'}

# Извлечение групп "глагол + прямое дополнение, выраженное существительным"

In [0]:
import logging

In [191]:
colloc = []
for i, tree in enumerate(trees):
  try:
    g = DependencyGraph(tree, top_relation_label='root')
    for t in g.triples():
      if t[0][1] == 'VERB' and t[1] == 'obj' and t[2][1] == 'NOUN':
        if t[0][0] in verbs:
          colloc.append((t[0][0], t[2][0]))
  except (AssertionError, TypeError):
    logging.error("Same old sufferings...", exc_info=True)

ERROR:root:Same old sufferings...
Traceback (most recent call last):
  File "<ipython-input-191-bc1ef357b6a6>", line 4, in <module>
    g = DependencyGraph(tree, top_relation_label='root')
  File "/usr/local/lib/python3.6/dist-packages/nltk/parse/dependencygraph.py", line 86, in __init__
    top_relation_label=top_relation_label,
  File "/usr/local/lib/python3.6/dist-packages/nltk/parse/dependencygraph.py", line 330, in _parse
    assert cell_number == len(cells)
AssertionError
ERROR:root:Same old sufferings...
Traceback (most recent call last):
  File "<ipython-input-191-bc1ef357b6a6>", line 4, in <module>
    g = DependencyGraph(tree, top_relation_label='root')
  File "/usr/local/lib/python3.6/dist-packages/nltk/parse/dependencygraph.py", line 86, in __init__
    top_relation_label=top_relation_label,
  File "/usr/local/lib/python3.6/dist-packages/nltk/parse/dependencygraph.py", line 330, in _parse
    assert cell_number == len(cells)
AssertionError
ERROR:root:Same old sufferings...


In [192]:
colloc 

[('признать', 'договор'),
 ('вынести', 'приговор'),
 ('признать', 'ответственность'),
 ('выплатить', 'компенсации'),
 ('обжаловать', 'санкцию'),
 ('удовлетворить', 'жалобу'),
 ('взыскать', 'деньги'),
 ('обжаловать', 'санкцию'),
 ('оспаривать', 'решение'),
 ('оспаривать', 'решение'),
 ('дать', 'возможность'),
 ('взыскать', 'задолженность'),
 ('обжаловать', 'решение'),
 ('получить', 'средства'),
 ('предъявить', 'обвинение'),
 ('получить', 'комментарии'),
 ('получить', 'комментарии'),
 ('обжаловать', 'решение'),
 ('дать', 'показания'),
 ('признать', 'организацию'),
 ('запретить', 'деятельность'),
 ('признать', 'организацию'),
 ('запретить', 'деятельность'),
 ('получить', 'компромат'),
 ('обязать', 'ответчиков'),
 ('получить', 'информацию'),
 ('вынести', 'приговор'),
 ('признать', 'право'),
 ('оспорить', 'решение'),
 ('рассматривать', 'дело'),
 ('доказать', 'Немцову'),
 ('использовать', 'возможности'),
 ('обжаловать', 'решение'),
 ('получить', 'деньги'),
 ('рассмотреть', 'возможность'),
 (

# Оценка коллокаций

In [0]:
import nltk.collocations
import nltk.corpus
import collections

In [0]:
lemmas = []
for tree in trees:
  for t in tree.split('\n'):
    line = t.split('\t')
    if len(line) > 3:
      if line[3] != 'PUNCT' and line[3] != 'CCONJ' and line[3] != 'PART':
        lemmas.append(line[2])

 log-likelihood

In [0]:
bgm = nltk.collocations.BigramAssocMeasures()

In [0]:
finder = nltk.collocations.BigramCollocationFinder.from_words(lemmas)

In [194]:
finder.nbest(bgm.likelihood_ratio, 20)

[('арбитражный', 'суд'),
 ('суд', 'Москва'),
 ('верховный', 'суд'),
 ('то', 'что'),
 ('уголовный', 'дело'),
 ('миллион', 'рубль'),
 ('2006', 'год'),
 ('лишение', 'свобода'),
 ('один', 'из'),
 ('миллиард', 'рубль'),
 ('2007', 'год'),
 ('2005', 'год'),
 ('тысяча', 'рубль'),
 ('2004', 'год'),
 ('под', 'стража'),
 ('Михаил', 'Ходорковский'),
 ('такой', 'образ'),
 ('ПрайсвотерхаусКуперс', 'Аудит'),
 ('по', 'дело'),
 ('Борис', 'Березовский')]

In [0]:
finder.apply_ngram_filter(lambda w1, w2: (w1, w2) not in colloc)

In [196]:
finder.score_ngrams(bgm.likelihood_ratio)

[(('подать', 'иск'), 384.99673536017997),
 (('удовлетворить', 'иск'), 363.0197891667802),
 (('принять', 'решение'), 292.2064347562424),
 (('удовлетворить', 'ходатайство'), 225.24803656624957),
 (('обжаловать', 'решение'), 202.75246166257),
 (('вынести', 'решение'), 202.71005415954647),
 (('иметь', 'право'), 186.8506393142544),
 (('просить', 'суд'), 170.90514590578346),
 (('вынести', 'приговор'), 134.12997771491533),
 (('предъявить', 'обвинение'), 115.53613290117497),
 (('отклонить', 'иск'), 109.25097528597927),
 (('отменить', 'решение'), 86.55441002786455),
 (('подтвердить', 'законность'), 79.17904125766024),
 (('отменить', 'постановление'), 73.3912281822325),
 (('оспаривать', 'решение'), 70.7578998562498),
 (('рассматривать', 'дело'), 60.06426883144751),
 (('рассматривать', 'иск'), 59.07819579912844),
 (('оспорить', 'предписание'), 52.5732858024067),
 (('рассмотреть', 'вопрос'), 49.55863459480531),
 (('оспаривать', 'законность'), 49.55668264900747),
 (('вынести', 'вердикт'), 48.691196

dice

In [121]:
finder.score_ngrams(bgm.dice)

[(('удовлетворить', 'ходатайство'), 0.17142857142857143),
 (('подать', 'иск'), 0.14628297362110312),
 (('удовлетворить', 'иск'), 0.14188267394270124),
 (('иметь', 'право'), 0.13529411764705881),
 (('подтвердить', 'законность'), 0.12598425196850394),
 (('оспаривать', 'законность'), 0.1111111111111111),
 (('оспорить', 'предписание'), 0.1111111111111111),
 (('предъявить', 'обвинение'), 0.1079136690647482),
 (('вынести', 'приговор'), 0.10734463276836158),
 (('отменить', 'постановление'), 0.09836065573770492),
 (('принять', 'решение'), 0.09056603773584905),
 (('вынести', 'решение'), 0.06981132075471698),
 (('рассмотреть', 'вопрос'), 0.06698564593301436),
 (('обжаловать', 'решение'), 0.06541129831516353),
 (('вынести', 'вердикт'), 0.06315789473684211),
 (('отменить', 'указ'), 0.057692307692307696),
 (('дать', 'возможность'), 0.05454545454545454),
 (('отклонить', 'иск'), 0.05396825396825397),
 (('оспорить', 'правомерность'), 0.05263157894736842),
 (('объявить', 'перерыв'), 0.05),
 (('подтверд

pmi

In [122]:
finder.score_ngrams(bgm.pmi)

[(('доказать', 'правность'), 11.184799235628002),
 (('получить', 'компромат'), 10.029004563020578),
 (('предъявить', 'ультиматум'), 9.388332629713137),
 (('оспорить', 'правомерность'), 8.930226408542406),
 (('доказать', 'неправомерность'), 8.86287114074064),
 (('объявить', 'перерыв'), 8.847764248350432),
 (('оспорить', 'предписание'), 8.79272288479247),
 (('удовлетворить', 'иски'), 8.58306102215602),
 (('оспаривать', 'законность'), 8.414804841074325),
 (('доказать', 'бездействие'), 8.377444313570399),
 (('подтвердить', 'законность'), 8.339969608151735),
 (('доказать', 'невиновность'), 8.014874234185692),
 (('отменить', 'указ'), 7.8352948963882145),
 (('удовлетворить', 'ходатайство'), 7.188201404814807),
 (('отменить', 'постановление'), 7.168718630113407),
 (('дать', 'возможность'), 7.151440638681571),
 (('вынести', 'вердикт'), 7.135810542192303),
 (('оспорить', 'доначисление'), 7.122871486484803),
 (('отменить', 'запрет'), 7.119087862388808),
 (('иметь', 'право'), 7.034757658516238),
 

# Золотой стандарт

In [0]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

In [0]:
best100_logl = finder.nbest(bgm.likelihood_ratio, 100)
best100_dice = finder.nbest(bgm.dice, 100)
best100_pmi = finder.nbest(bgm.pmi, 100)

In [0]:
gold = set(best100_logl).intersection(set(best100_dice), set(best100_pmi))

In [197]:
gold

{('взыскать', 'деньги'),
 ('вынести', 'вердикт'),
 ('вынести', 'приговор'),
 ('вынести', 'решение'),
 ('вынести', 'суд'),
 ('выплатить', 'деньги'),
 ('выплатить', 'штраф'),
 ('дать', 'возможность'),
 ('дать', 'заключение'),
 ('дать', 'определение'),
 ('дать', 'указание'),
 ('делать', 'вид'),
 ('доказать', 'бездействие'),
 ('доказать', 'невиновность'),
 ('доказать', 'незаконность'),
 ('доказать', 'неправомерность'),
 ('доказать', 'правность'),
 ('запретить', 'ЗАО'),
 ('запретить', 'деятельность'),
 ('иметь', 'повод'),
 ('иметь', 'право'),
 ('назначить', 'наказание'),
 ('направить', 'запрос'),
 ('обжаловать', 'вердикт'),
 ('обжаловать', 'приговор'),
 ('обжаловать', 'решение'),
 ('объявить', 'перерыв'),
 ('оспаривать', 'законность'),
 ('оспаривать', 'отказ'),
 ('оспаривать', 'решение'),
 ('оспорить', 'выборы'),
 ('оспорить', 'доначисление'),
 ('оспорить', 'правомерность'),
 ('оспорить', 'предписание'),
 ('оспорить', 'решение'),
 ('оспорить', 'указ'),
 ('оставить', 'иск'),
 ('оставить', 'п

In [0]:
with open('verb_coll.txt', 'r', encoding = 'utf-8') as f:
  lines = f.readlines()

In [129]:
lines

['аборт\tbase=действие mood=объект order=NV\tаборт делать\n',
 'аборт\tbase=действие mood=объект order=VN\tделать аборт\n',
 'аборт\tbase=действие mood=объект order=VN\tсделать аборт\n',
 'авария\tbase=существование/создание mood=субъект semclass=факт order=NV\tавария произошла\n',
 'авария\tbase=существование/создание mood=субъект semclass=факт order=NV\tавария случилась\n',
 'авария\tbase=действие mood=объект order=VN\tпотерпеть аварию\n',
 'авария\tbase=существование/создание mood=субъект semclass=факт order=VN\tпроизошла авария\n',
 'авария\tbase=существование/создание mood=субъект semclass=факт order=VN\tслучилась авария\n',
 'авторитет\tbase=обладание phase=начало order=VN\tзавоевать авторитет\n',
 'авторитет\tsemclass=содействие subclass=вред order=VN\tподорвать авторитет\n',
 'авторитет\tsemclass=содействие subclass=вред order=VN\tподорвать авторитет\n',
 'авторитет\tbase=обладание mood=непрямой объект order=VN\tпользоваться авторитетом\n',
 'авторитет\tbase=обладание mood=непр

In [0]:
lemmas_from_dict = []
for line in lines:
  line = line.split('\t') 
  if len(line) > 2:
    colloc = line[2].split()
    if len(colloc) == 2:      
      lemmas_ = (m.parse(colloc[0])[0].normal_form, m.parse(colloc[1])[0].normal_form)
      lemmas_from_dict.append(lemmas_)

In [162]:
lemmas_from_dict

[('аборт', 'делать'),
 ('делать', 'аборт'),
 ('сделать', 'аборт'),
 ('авария', 'произойти'),
 ('авария', 'случиться'),
 ('потерпеть', 'авария'),
 ('произойти', 'авария'),
 ('случиться', 'авария'),
 ('завоевать', 'авторитет'),
 ('подорвать', 'авторитет'),
 ('подорвать', 'авторитет'),
 ('пользоваться', 'авторитет'),
 ('оспаривать', 'акт'),
 ('активность', 'проявлять'),
 ('активность', 'проявлять'),
 ('проявить', 'активность'),
 ('проявить', 'активность'),
 ('проявлять', 'активность'),
 ('проявлять', 'активность'),
 ('акцент', 'делаться'),
 ('акцент', 'переноситься'),
 ('акцент', 'сделать'),
 ('делать', 'акцент'),
 ('расставлять', 'акцент'),
 ('расставлять', 'акцент'),
 ('расставлять', 'акцент'),
 ('сделать', 'акцент'),
 ('сместить', 'акцент'),
 ('сместить', 'акцент'),
 ('предлагать', 'алгоритм'),
 ('предлагать', 'алгоритм'),
 ('разработать', 'алгоритм'),
 ('разработать', 'алгоритм'),
 ('вызывать', 'аллергия'),
 ('вызывать', 'аллергия'),
 ('найти', 'альтернатива'),
 ('найти', 'альтернатив

In [0]:
gold_new = gold.intersection(set(lemmas_from_dict))

In [165]:
gold_new

{('вынести', 'приговор'),
 ('вынести', 'решение'),
 ('дать', 'возможность'),
 ('дать', 'заключение'),
 ('дать', 'определение'),
 ('дать', 'указание'),
 ('делать', 'вид'),
 ('направить', 'запрос'),
 ('обжаловать', 'приговор'),
 ('обжаловать', 'решение'),
 ('объявить', 'перерыв'),
 ('оспаривать', 'решение'),
 ('отменить', 'постановление'),
 ('отменить', 'приговор'),
 ('отменить', 'решение'),
 ('передать', 'дело'),
 ('подать', 'иск'),
 ('получить', 'контроль'),
 ('получить', 'срок'),
 ('предъявить', 'иск'),
 ('предъявить', 'обвинение'),
 ('признать', 'право'),
 ('принять', 'решение'),
 ('рассматривать', 'вопрос'),
 ('рассматривать', 'дело'),
 ('рассмотреть', 'возможность'),
 ('рассмотреть', 'вопрос'),
 ('решить', 'вопрос'),
 ('удовлетворить', 'запрос'),
 ('удовлетворить', 'иск')}

# Сочетания, не вошедшие в словарь, но являющиеся коллокациями

Не вошедшее

In [0]:
missing = gold.difference(set(lemmas_from_dict))

In [168]:
missing

{('взыскать', 'деньги'),
 ('вынести', 'вердикт'),
 ('вынести', 'суд'),
 ('выплатить', 'деньги'),
 ('выплатить', 'штраф'),
 ('доказать', 'бездействие'),
 ('доказать', 'невиновность'),
 ('доказать', 'незаконность'),
 ('доказать', 'неправомерность'),
 ('доказать', 'правность'),
 ('запретить', 'ЗАО'),
 ('запретить', 'деятельность'),
 ('иметь', 'повод'),
 ('иметь', 'право'),
 ('назначить', 'наказание'),
 ('обжаловать', 'вердикт'),
 ('оспаривать', 'законность'),
 ('оспаривать', 'отказ'),
 ('оспорить', 'выборы'),
 ('оспорить', 'доначисление'),
 ('оспорить', 'правомерность'),
 ('оспорить', 'предписание'),
 ('оспорить', 'решение'),
 ('оспорить', 'указ'),
 ('оставить', 'иск'),
 ('оставить', 'приговор'),
 ('отклонить', 'иск'),
 ('отменить', 'арест'),
 ('отменить', 'запрет'),
 ('отменить', 'указ'),
 ('подать', 'ходатайство'),
 ('подтвердить', 'законность'),
 ('подтвердить', 'факт'),
 ('получить', 'деньги'),
 ('получить', 'компромат'),
 ('предъявить', 'ультиматум'),
 ('признать', 'договор'),
 ('при

Добавим в ЗС

In [0]:
gold_new = gold_new.union(missing)

In [198]:
gold_new

{('взыскать', 'деньги'),
 ('вынести', 'вердикт'),
 ('вынести', 'приговор'),
 ('вынести', 'решение'),
 ('вынести', 'суд'),
 ('выплатить', 'деньги'),
 ('выплатить', 'штраф'),
 ('дать', 'возможность'),
 ('дать', 'заключение'),
 ('дать', 'определение'),
 ('дать', 'указание'),
 ('делать', 'вид'),
 ('доказать', 'бездействие'),
 ('доказать', 'невиновность'),
 ('доказать', 'незаконность'),
 ('доказать', 'неправомерность'),
 ('доказать', 'правность'),
 ('запретить', 'ЗАО'),
 ('запретить', 'деятельность'),
 ('иметь', 'повод'),
 ('иметь', 'право'),
 ('назначить', 'наказание'),
 ('направить', 'запрос'),
 ('обжаловать', 'вердикт'),
 ('обжаловать', 'приговор'),
 ('обжаловать', 'решение'),
 ('объявить', 'перерыв'),
 ('оспаривать', 'законность'),
 ('оспаривать', 'отказ'),
 ('оспаривать', 'решение'),
 ('оспорить', 'выборы'),
 ('оспорить', 'доначисление'),
 ('оспорить', 'правомерность'),
 ('оспорить', 'предписание'),
 ('оспорить', 'решение'),
 ('оспорить', 'указ'),
 ('оставить', 'иск'),
 ('оставить', 'п

##Ранговая корреляция (коэффициент Спирмана)

In [0]:
from scipy import stats

In [0]:
#50 - для примера

logl_best = finder.nbest(bgm.likelihood_ratio, 50) 
dice_best = finder.nbest(bgm.dice, 50)
pmi_best = finder.nbest(bgm.pmi, 50)

In [0]:
finder2 = nltk.collocations.BigramCollocationFinder.from_words(lemmas)
finder2.apply_ngram_filter(lambda w1, w2: (w1, w2) not in colloc)
finder2.apply_ngram_filter(lambda w1, w2: (w1, w2) not in gold_new)

In [0]:
logl_best_gold = finder2.nbest(bgm.likelihood_ratio, 50)
dice_best_gold = finder2.nbest(bgm.dice, 50)
pmi_best_gold = finder2.nbest(bgm.pmi, 50)

log-likelihood

In [204]:
stats.spearmanr(logl_best, logl_best_gold)

SpearmanrResult(correlation=array([[1.        , 0.08718159, 1.        , 0.08718159],
       [0.08718159, 1.        , 0.08718159, 1.        ],
       [1.        , 0.08718159, 1.        , 0.08718159],
       [0.08718159, 1.        , 0.08718159, 1.        ]]), pvalue=array([[0.        , 0.54715899, 0.        , 0.54715899],
       [0.54715899, 0.        , 0.54715899, 0.        ],
       [0.        , 0.54715899, 0.        , 0.54715899],
       [0.54715899, 0.        , 0.54715899, 0.        ]]))

dice

In [205]:
stats.spearmanr(dice_best, dice_best_gold)

SpearmanrResult(correlation=array([[1.        , 0.11886007, 1.        , 0.11886007],
       [0.11886007, 1.        , 0.11886007, 1.        ],
       [1.        , 0.11886007, 1.        , 0.11886007],
       [0.11886007, 1.        , 0.11886007, 1.        ]]), pvalue=array([[0.        , 0.41100288, 0.        , 0.41100288],
       [0.41100288, 0.        , 0.41100288, 0.        ],
       [0.        , 0.41100288, 0.        , 0.41100288],
       [0.41100288, 0.        , 0.41100288, 0.        ]]))

pmi

In [206]:
stats.spearmanr(pmi_best, pmi_best_gold)

SpearmanrResult(correlation=array([[1.        , 0.14385054, 1.        , 0.14385054],
       [0.14385054, 1.        , 0.14385054, 1.        ],
       [1.        , 0.14385054, 1.        , 0.14385054],
       [0.14385054, 1.        , 0.14385054, 1.        ]]), pvalue=array([[0.        , 0.31893671, 0.        , 0.31893671],
       [0.31893671, 0.        , 0.31893671, 0.        ],
       [0.        , 0.31893671, 0.        , 0.31893671],
       [0.31893671, 0.        , 0.31893671, 0.        ]]))